In [1]:
import os  
import pdfplumber
import numpy as np
import pandas as pd
from WindPy import w


In [2]:

w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒 

w.isconnected() # 判断WindPy是否已经登录成功


Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


True

In [83]:
#改金额单位
def mc(amount):
    value = ''
    amount = float(amount)
    if abs(amount) >= 10000 and abs(amount) < 100000000:
        val = round(amount/10000)
        value = str(val) + "万元"
    elif abs(amount) >= 100000000:
        val = round(amount/100000000,2)
        value = str(val) + "亿元"
        
    return value 

In [4]:
def file_name(file_dir):  
    L=[]  
    for root, dirs, files in os.walk(file_dir): 
        for file in files: 
            if os.path.splitext(file)[1] == '.pdf': 
                L.append(os.path.join(root, file)) 
    return L


In [5]:
# 读取每一个pdf的股票代码
dir_name = 'D:/GF/time_saver/半年度报告生成器'
# dir_list = os.listdir(dir_name)
# print(dir_list[1])
filelist = file_name(dir_name)


tot_num_files = np.size(filelist)
code = {}
#
for i in range(0,tot_num_files-1):
    if filelist[i].find('.S')!= -1 :
        a = filelist[i].find('.S') + 3
        b = filelist[i][:a]
        c = b[(a-9):]
        code[i] = c
        

        


filelist

['D:/GF/time_saver/半年度报告生成器\\20190821-300085.SZ-银之杰：2019年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-002296.SZ-辉煌科技_2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-600519.SH-600519贵州茅台2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-07-29-688396.SH-688396华润微2020年半年度报告.pdf',
 'D:/GF/time_saver/半年度报告生成器\\2020-08-04-002439.SZ-启明星辰_2020年半年度报告.pdf']

In [6]:
filename = "2020-08-04-002439.SZ-启明星辰_2020年半年度报告.pdf"
code = input("股票代码：")
pdf = pdfplumber.open(filename)


股票代码：002439.SZ


In [7]:
############################ 主要会计数据和财务指标 ###############################

# 自动查找页码

for i in range(2,10):
    p0 = pdf.pages[i]
    pp = p0.extract_text(x_tolerance=3, y_tolerance=3)
    if pp.find("主要会计数据")> -1:
        page_num = i
        break
        
        

In [8]:


p0 = pdf.pages[page_num]
table = p0.extract_table()


In [9]:
table

[['', '本报告期', '上年同期', '本报告期比上年同期增减'],
 ['营业收入（元）', '754,588,255.05', '882,092,643.61', '-14.45%'],
 ['归属于上市公司股东的净利润（元）', '-21,382,841.09', '13,754,173.09', '-255.46%'],
 ['归属于上市公司股东的扣除非经常性损', '', '', ''],
 [None, '-50,092,088.91', '-2,102,363.16', '-2,282.66%'],
 ['益的净利润（元）', None, None, None],
 [None, '', '', ''],
 ['经营活动产生的现金流量净额（元）', '-71,172,687.72', '-136,185,017.18', '47.74%'],
 ['基本每股收益（元/股）', '-0.0230', '0.0150', '-253.33%'],
 ['稀释每股收益（元/股）', '-0.0230', '0.0150', '-253.33%'],
 ['加权平均净资产收益率', '-0.45%', '0.38%', '-0.83%'],
 ['', '本报告期末', '上年度末', '本报告期末比上年度末增减'],
 ['总资产（元）', '6,521,686,579.92', '6,896,155,020.04', '-5.43%'],
 ['归属于上市公司股东的净资产（元）', '5,180,258,181.28', '4,363,298,134.45', '18.72%']]

In [10]:

df = pd.DataFrame(table[1:6], columns=table[0])
# for column in ["Effective", "Received"]:
#     df[column] = df[column].str.replace(" ", "")
df



,,本报告期,上年同期,本报告期比上年同期增减
0,营业收入（元）,"754,588,255.05","882,092,643.61",-14.45%
1,归属于上市公司股东的净利润（元）,"-21,382,841.09","13,754,173.09",-255.46%
2,归属于上市公司股东的扣除非经常性损,,,
3,None,"-50,092,088.91","-2,102,363.16","-2,282.66%"
4,益的净利润（元）,None,None,None


In [11]:
# 洗数据。。。
data = df.values

#设置变量index，保证扣非的读取
test_var1 = data[3,0]
test_var2 = data[2,0]
index = []
if test_var1!= None and test_var2.find("经常性")!= -1:
    index = int(2)
elif test_var1==None and test_var2.find("经常性")!= -1:
    index = int(3)
    
    
percent = {}

#去除逗号
for i in [0,1,index]:
    for j in [1,2,3]:
        data[i,j] = data[i,j].replace(',','')
        percent[i] = data[i,3]
        if percent[i].find('%')!=-1:
            percent[i] = percent[i].replace('%','')
        



In [12]:
# 修改数据类型
percent[0] = float(percent[0])
percent[1] = float(percent[1])
percent[index] = float(percent[index])

# 判断增减
updown = {}
for i in [0,1,index]:
    if percent[i]>=0:
        updown[i]="增加"
    else:
        updown[i]="减少"
        percent[i]=abs(percent[i])



In [13]:
################################ 营业利润 #########################################
# 从合并利润表里提取
page_num1 = 0
for z in range(20,80):
    p1 = pdf.pages[z]
    pp1 = p1.extract_text(x_tolerance=3, y_tolerance=3)
    if pp1.find("三、营业利润（亏损")> -1:
        page_num1 = z
        break
        

# 提取表格内容
p1 = pdf.pages[page_num1]
table1 = p1.extract_table()
table1


[['三、营业利润（亏损以“－”号填列）', '-67,238,150.79', '-27,817,234.63'],
 ['加：营业外收入', '961,217.01', '925,019.25'],
 ['减：营业外支出', '937,706.76', '269,491.33'],
 ['四、利润总额（亏损总额以“－”号填列）', '-67,214,640.54', '-27,161,706.71'],
 ['减：所得税费用', '-43,325,850.72', '-35,152,406.63'],
 ['五、净利润（净亏损以“－”号填列）', '-23,888,789.82', '7,990,699.92'],
 ['（一）按经营持续性分类', '', ''],
 ['1.持续经营净利润（净亏损以“－”号填列）', '-23,888,789.82', '7,990,699.92'],
 ['2.终止经营净利润（净亏损以“－”号填列）', '', ''],
 ['（二）按所有权归属分类', '', ''],
 ['1.归属于母公司所有者的净利润', '-21,382,841.09', '13,754,173.09'],
 ['2.少数股东损益', '-2,505,948.73', '-5,763,473.17'],
 ['六、其他综合收益的税后净额', '2,067,500.15', '178,122.36'],
 ['归属母公司所有者的其他综合收益的税后净额', '2,067,500.15', '178,122.36'],
 ['（一）不能重分类进损益的其他综合收益', '', ''],
 ['1.重新计量设定受益计划变动额', '', ''],
 ['2.权益法下不能转损益的其他综合收益', '', ''],
 ['3.其他权益工具投资公允价值变动', '', ''],
 ['4.企业自身信用风险公允价值变动', '', ''],
 ['5.其他', '', ''],
 ['（二）将重分类进损益的其他综合收益', '2,067,500.15', '178,122.36'],
 ['1.权益法下可转损益的其他综合收益', '', ''],
 ['2.其他债权投资公允价值变动', '', ''],
 ['3.金融资产重分类计入其他综合收益的金额', '', '

In [14]:
# 把表格装进dataframe
df1 = pd.DataFrame(table1, columns=table1[0])

df1


,三、营业利润（亏损以“－”号填列）,"-67,238,150.79","-27,817,234.63"
0,三、营业利润（亏损以“－”号填列）,"-67,238,150.79","-27,817,234.63"
1,加：营业外收入,"961,217.01","925,019.25"
2,减：营业外支出,"937,706.76","269,491.33"
3,四、利润总额（亏损总额以“－”号填列）,"-67,214,640.54","-27,161,706.71"
4,减：所得税费用,"-43,325,850.72","-35,152,406.63"
5,五、净利润（净亏损以“－”号填列）,"-23,888,789.82","7,990,699.92"
6,（一）按经营持续性分类,,
7,1.持续经营净利润（净亏损以“－”号填列）,"-23,888,789.82","7,990,699.92"
8,2.终止经营净利润（净亏损以“－”号填列）,,
9,（二）按所有权归属分类,,


In [15]:
# 还是洗数据

#查找营业利润在第几行：

dfsize = df1.count()[0]

for i in range(0,dfsize-1):
    if df1.iloc[i][0] == "三、营业利润（亏损以“－”号填列）":
        loc = i
        break

# 读取该行数据
# 分离附注

profit_this_temp = df1.iloc[loc][1]
profit_last_temp = df1.iloc[loc][2]

# 去逗号
profit_this = float(profit_this_temp.replace(',',''))
profit_last = float(profit_last_temp.replace(',',''))

# 计算增减比率
profit_var = round(100*(profit_this - profit_last)/profit_last,2)


# 判断增减

if profit_var > 0:
    updown_p = '增加'
elif profit_var < 0:
    updown_p = '减少'

#特殊情况
if profit_this < 0 and profit_this < 0:
    updown_p = '减少'
    
    

In [16]:
Revenue = {'obj':data[0,0],'this':data[0,1],'last':data[0,2],'varpct':percent[0]} #把数据写入dict

NetProfit_sh = {'obj':data[1,0],'this':data[1,1],'last':data[1,2],'varpct':percent[1]} #把数据写入dict

NetProfit_sh_loss = {'obj':data[2,0],'this':data[index,1],'last':data[index,2],'varpct':percent[index]} #把数据写入dict

Profit = {'obj':df1.iloc[loc][0],'this':profit_this,'last':profit_last,'varpct':profit_var}




In [17]:
# 上半年生成
line1_h1 = "20H1营业总收入" + mc(Revenue['this']) + ",上年同期为" + mc(Revenue['last']) + ",同比" + updown[0] + str(percent[0]) + '%'
line2_h1 = "20H1营业利润" + mc(Profit['this']) + ",上年同期为" + mc(Profit['last']) + ",同比" + updown_p + str(Profit['varpct']) + '%'
line3_h1 = "20H1归母净利润" + mc(NetProfit_sh['this']) + ",上年同期为" + mc(NetProfit_sh['last']) + ",同比" + updown[1] + str(percent[1]) + '%'
line4_h1 = "20H1扣非归母净利润" + mc(NetProfit_sh_loss['this']) + ",上年同期为" + mc(NetProfit_sh_loss['last']) + ",同比" + updown[index] + str(percent[index]) + '%'

In [18]:
######### 以下是wind部分 ##########

In [19]:
#营业总收入 _y
# 从wind查找q1数据

wsddata1 = w.wsd(code, "tot_oper_rev", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata1


.ErrorCode=0
.Codes=[002439.SZ]
.Fields=[TOT_OPER_REV]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[347101346.28,882092643.61,1582941789.86,3089495530.2,229730569.24]]

In [20]:
#计算q2数据
lastq1_y = wsddata1.Data[0][0] #选取2019年q1数据
lasth1_y = wsddata1.Data[0][1] #选取2019年h1数据
thisq1_y = wsddata1.Data[0][4] #选取2020年q1数据
thish1_y = float(Revenue['this'])
lastq2_y = lasth1_y - lastq1_y    #计算2019年q2数据
thisq2_y = thish1_y - thisq1_y 

#计算同比增长/减少
growth_year = {}
growth_year[0] = (thisq2_y - lastq2_y)/abs(lastq2_y) * 100

In [21]:
#归母净利润 _g
# 从wind查找q1数据
wsddata2 = w.wsd(code, "np_belongto_parcomsh", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata2



.ErrorCode=0
.Codes=[002439.SZ]
.Fields=[NP_BELONGTO_PARCOMSH]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-40511495.65,13754173.09,97095086.44,688277790.16,-79348325.49]]

In [22]:

#计算q2数据
lastq1_g = wsddata2.Data[0][0] #选取2019年q1数据
lasth1_g = wsddata2.Data[0][1] #选取2019年h1数据
thisq1_g = wsddata2.Data[0][4] #选取2020年q1数据
thish1_g = float(NetProfit_sh['this'])
lastq2_g = lasth1_g - lastq1_g     #计算2019年q2数据
thisq2_g = thish1_g - thisq1_g 

#计算同比增长/减少
growth_year[1] = (thisq2_g - lastq2_g)/abs(lastq2_g) * 100

In [23]:
#扣非归母净利润 _k
# 从wind查找q1数据
wsddata3 = w.wsd(code, "deductedprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata3


.ErrorCode=0
.Codes=[002439.SZ]
.Fields=[DEDUCTEDPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-44887040.63,-2102363.16,77652730.29,574449335.1,-90790020.04]]

In [24]:
#计算q2数据
lastq1_k = wsddata3.Data[0][0] #选取2019年q1数据
lasth1_k = wsddata3.Data[0][1] #选取2019年h1数据
thisq1_k = wsddata3.Data[0][4] #选取2020年q1数据
thish1_k = float(NetProfit_sh_loss['this'])
lastq2_k = lasth1_k - lastq1_k     #计算2019年q2数据
thisq2_k = thish1_k - thisq1_k 

#计算同比增长/减少
growth_year[2] = (thisq2_k - lastq2_k)/abs(lastq2_k) * 100

In [25]:
#营业利润 _lr
# 从wind查找q1数据
wsddata4 = w.wsd(code, "opprofit", "2019-01-01", "2020-03-31", "rptType=1;Period=Q;Days=Alldays;Fill=Previous")
wsddata4


.ErrorCode=0
.Codes=[002439.SZ]
.Fields=[OPPROFIT]
.Times=[20190331,20190630,20190930,20191231,20200331]
.Data=[[-56354543.92,-27817234.63,51716113.82,740658305.56,-106668418.86]]

In [26]:
#计算q2数据
lastq1_lr = wsddata4.Data[0][0] #选取2019年q1数据
lasth1_lr = wsddata4.Data[0][1] #选取2019年h1数据
thisq1_lr = wsddata4.Data[0][4] #选取2020年q1数据
thish1_lr = float(Profit['this'])
lastq2_lr = lasth1_lr - lastq1_lr     #计算2019年q2数据
thisq2_lr = thish1_lr - thisq1_lr 

#计算同比增长/减少
growth_year_p = (thisq2_lr - lastq2_lr)/abs(lastq2_lr) * 100

In [27]:
# 判断增减
updown_q2 = {}
for i in [0,1,2]:
    if growth_year[i]>=0:
        updown_q2[i]="增加"
    else:
        updown_q2[i]="减少"
        growth_year[i]=abs(growth_year[i])

if growth_year_p >=0:
    updown_p_q2="增加"
else:
    updown_p_q2="减少"



In [28]:
# 二季度生成
line1_q2 = "20Q2营业总收入" + mc(thisq2_y) + ",上年同期为" + mc(lastq2_y) + ",同比" + updown_q2[0] + str(round(growth_year[0],2)) + '%'
line2_q2 = "20Q2营业利润" + mc(thisq2_lr) + ",上年同期为" + mc(lastq2_lr) + ",同比" + updown_p_q2 + str(round(growth_year_p,2)) + '%'
line3_q2 = "20Q2归母净利润" + mc(thisq2_g) + ",上年同期为" + mc(lastq2_g) + ",同比" + updown_q2[1] + str(round(growth_year[1],2)) + '%'
line4_q2 = "20Q2扣非归母净利润" + mc(thisq2_k) + ",上年同期为" + mc(lastq2_k) + ",同比" + updown_q2[2] + str(round(growth_year[2],2)) + '%'

In [30]:
############################### 费用 ################################

# 从合并利润表里提取
page_num2 = 0
for y in range(20,80):
    p2 = pdf.pages[y]
    pp2 = p2.extract_text(x_tolerance=3, y_tolerance=3)
    if pp2.find("研发费用")> -1:
        page_num2 = y
        break
        

# 提取表格内容
p2 = pdf.pages[page_num2]
table2 = p2.extract_table()
table2


[['项目', '2020年半年度', '2019年半年度'],
 ['一、营业总收入', '754,588,255.05', '882,092,643.61'],
 ['其中：营业收入', '754,588,255.05', '882,092,643.61'],
 ['利息收入', '', ''],
 ['已赚保费', '', ''],
 ['手续费及佣金收入', '', ''],
 ['二、营业总成本', '928,890,045.40', '956,079,804.28'],
 ['其中：营业成本', '216,624,449.68', '322,184,922.11'],
 ['利息支出', '', ''],
 ['手续费及佣金支出', '', ''],
 ['退保金', '', ''],
 ['赔付支出净额', '', ''],
 ['提取保险责任准备金净额', '', ''],
 ['保单红利支出', '', ''],
 ['分保费用', '', ''],
 ['税金及附加', '13,702,795.68', '15,589,974.62'],
 ['销售费用', '304,030,609.25', '263,402,693.61'],
 ['管理费用', '69,618,185.29', '64,134,553.84'],
 ['研发费用', '322,948,773.33', '284,194,460.67'],
 ['财务费用', '1,965,232.17', '6,573,199.43'],
 ['其中：利息费用', '7,801,304.17', '11,009,815.65'],
 ['利息收入', '5,422,084.61', '4,486,383.70'],
 ['加：其他收益', '102,377,342.01', '62,233,512.17'],
 ['投资收益（损失以“－”号填列）', '19,490,240.78', '79,405.17'],
 ['其中：对联营企业和合营企业的投资收益', '-749,754.93', '-3,312,988.27'],
 ['以摊余成本计量的金融资产终止确认收益', '', ''],
 ['汇兑收益（损失以“-”号填列）', '', ''],
 ['净敞口套期收益（损失以“－”号填列）

In [31]:
# 把表格装进dataframe
df2 = pd.DataFrame(table2, columns=table1[0])

df2


,三、营业利润（亏损以“－”号填列）,"-67,238,150.79","-27,817,234.63"
0,项目,2020年半年度,2019年半年度
1,一、营业总收入,"754,588,255.05","882,092,643.61"
2,其中：营业收入,"754,588,255.05","882,092,643.61"
3,利息收入,,
4,已赚保费,,
5,手续费及佣金收入,,
6,二、营业总成本,"928,890,045.40","956,079,804.28"
7,其中：营业成本,"216,624,449.68","322,184,922.11"
8,利息支出,,
9,手续费及佣金支出,,


In [32]:
# 还是洗数据

#查找费用在第几行：

dfsize2 = df2.count()[0]

for i in range(0,dfsize2-1):
    if df2.iloc[i][0] == "销售费用":
        loc2 = i
        break

In [84]:
# 填写数据+去逗号
mktex_this = float(df2.iloc[loc2][1].replace(',',''))
adminex_this = float(df2.iloc[loc2+1][1].replace(',',''))
rndex_this = float(df2.iloc[loc2+2][1].replace(',',''))
finex_this = float(df2.iloc[loc2+3][1].replace(',',''))

mktex_last = float(df2.iloc[loc2][2].replace(',',''))
adminex_last = float(df2.iloc[loc2+1][2].replace(',',''))
rndex_last = float(df2.iloc[loc2+2][2].replace(',',''))
finex_last = float(df2.iloc[loc2+3][2].replace(',',''))

# 计算同比增减

mktex_var = round(100*(mktex_this - mktex_last)/mktex_last,2)
adminex_var = round(100*(adminex_this - adminex_last)/adminex_last,2)
rndex_var = round(100*(rndex_this - rndex_last)/rndex_last,2)
finex_var = round(100*(finex_this - finex_last)/finex_last,2)


# 判断增减
updown_ex={}
numm = 0
for i in [mktex_var,adminex_var,rndex_var,finex_var]:    
    if i > 0:
        updown_ex[numm] = '增加'
    elif i < 0:
        updown_ex[numm] = '减少'
    numm = numm+1


# 计算今年的费用率
mktex_rate_this = round(100*mktex_this/float(Revenue['this']),2)
adminex_rate_this = round(100*adminex_this/float(Revenue['this']),2)
rndex_rate_this = round(100*rndex_this/float(Revenue['this']),2)
finex_rate_this = round(100*finex_this/float(Revenue['this']),2)

# 计算去年的费用率
mktex_rate_last = round(100*mktex_last/float(Revenue['last']),2)
adminex_rate_last = round(100*adminex_last/float(Revenue['last']),2)
rndex_rate_last = round(100*rndex_last/float(Revenue['last']),2)
finex_rate_last = round(100*finex_last/float(Revenue['last']),2)

# 计算差值
mktex_rate_diff = round(mktex_rate_this - mktex_rate_last,2)
adminex_rate_diff = round(adminex_rate_this - adminex_rate_last,2)
rndex_rate_diff = round(rndex_rate_this - rndex_rate_last,2)
finex_rate_diff = round(finex_rate_this - finex_rate_last,2)

# 判断较去年同期增减
updown_rate={}
nu = 0
for i in [mktex_rate_diff,adminex_rate_diff,rndex_rate_diff,finex_rate_diff]:    
    if i > 0:
        updown_rate[nu] = '增加'
    elif i < 0:
        updown_rate[nu] = '减少'
    nu = nu+1


In [85]:
rateline1 = "销售费用" + mc(mktex_this) + "，同比" + updown_ex[0] + str(abs(mktex_var)) + '%' +"；销售费用率"+ str(mktex_rate_this)+'%'+"，较去年同期"+ updown_rate[0] + str(abs(mktex_rate_diff)) + 'pct；'
rateline2 = "管理费用" + mc(adminex_this) + "，同比" + updown_ex[1] + str(abs(adminex_var)) + '%' +"；管理费用率"+ str(adminex_rate_this)+'%'+"，较去年同期"+ updown_rate[1] + str(abs(adminex_rate_diff)) + 'pct；'
rateline3 = "研发费用" + mc(rndex_this) + "，同比" + updown_ex[2] + str(abs(rndex_var)) + '%' +"；研发费用率"+ str(rndex_rate_this)+'%'+"，较去年同期"+ updown_rate[2] + str(abs(rndex_rate_diff)) + 'pct；'
rateline4 = "财务费用" + mc(finex_this) + "，同比" + updown_ex[3] + str(abs(finex_var)) + '%' +"；财务费用率"+ str(finex_rate_this)+'%'+"，较去年同期"+ updown_rate[3] + str(abs(finex_rate_diff)) + 'pct。'

In [92]:
# 财务摘要输出
print(line1_h1)
print(line2_h1)
print(line3_h1)
print(line4_h1)
print(line1_q2)
print(line2_q2)
print(line3_q2)
print(line4_q2)
print('\n')
#费用输出
print(rateline1)
print(rateline2)
print(rateline3)
print(rateline4)
print('\n')

20H1营业总收入7.55亿元,上年同期为8.82亿元,同比减少14.45%
20H1营业利润-6723.82万元,上年同期为-2781.72万元,同比减少141.71%
20H1归母净利润-2138.28万元,上年同期为1375.42万元,同比减少255.46%
20H1扣非归母净利润-5009.21万元,上年同期为-210.24万元,同比减少2282.66%
20Q2营业总收入5.25亿元,上年同期为5.35亿元,同比减少1.89%
20Q2营业利润3943.03万元,上年同期为2853.73万元,同比增加38.17%
20Q2归母净利润5796.55万元,上年同期为5426.57万元,同比增加6.82%
20Q2扣非归母净利润4069.79万元,上年同期为4278.47万元,同比减少4.88%


销售费用3.04亿元，同比增加15.42%；销售费用率40.29%，较去年同期增加10.43pct；
管理费用6962万元，同比增加8.55%；管理费用率9.23%，较去年同期增加1.96pct；
研发费用3.23亿元，同比增加13.64%；研发费用率42.8%，较去年同期增加10.58pct；
财务费用197万元，同比减少70.1%；财务费用率0.26%，较去年同期减少0.49pct。




In [35]:
【广发计算机刘雪峰团队-启明星辰（002439）2020年半年度报告】
20H1：
️ 营业总收入7.55亿元，同比减少14.5%，去年同期为8.82亿元；
️ 营业利润-6724万元，同比减少141.71%，去年同期为-2782万元；
️ 归母净利-2138万元，同比减少255.46%，去年同期为1375万元；
️ 归母扣非净利-5009万元，同比减少2282.66%，去年同期为-210万元。

20Q2：
️ 营业总收入5.25亿元，同比减少1.89%，去年同期为5.35亿元；
️ 营业利润3943万元，同比增长38.17%，去年同期为2854万元；
️ 归母净利5797万元，同比增长6.82%，去年同期为5427万元；
️ 归母扣非净利4070万元，同比减少4.88%，去年同期为4278万元。
 
分产品：
⭐️ 安全产品收入5.11亿元，同比减少24.57%；毛利率71.98%，较去年同期上升11.68pct；
⭐️ 安全运营与服务收入2.35亿元，同比增加21.14%；毛利率70.01%，较去年同期下降3.99pct；
⭐️ 其他业务收入802万元，同比减少22.11%；毛利率65.29%，较去年同期下降8.33pct；
 
费用：
️ 销售费用3.04亿元，同比增加15.4%；销售费用率40.3%，较去年同期增加10.4pct；
️ 管理费用6962万元，同比增加8.6%；管理费用率9.2%，较去年同期增加2pct；
️ 财务费用197万元，同比减少70.1%；财务费用率0.26%，较去年同期减少0.48pct；
️ 研发费用3.23亿元，同比增加13.6%；研发费用率为42.80%，较去年同期增加10.58pct，研发支出资本化率1.9%。
 
资产与负债：
⭐️ 应收账款16.43亿元，同比增加14.79%；
⭐️ 预收款项及合同负债2.41亿元，同比增长27%。
 
现金流：
️ 经营活动现金净流出7117万元，净流出同比减少47.74%，去年同期净流出1.36亿元；
️ 投资活动现金净流出3.93亿元，净流出同比增加5010.85%，去年同期净流入801万元；
️ 筹资活动现金净流出2375万元，净流出同比增加102.52%，去年同期净流入9.42亿元。

SyntaxError: invalid character in identifier (<ipython-input-35-a1b1803b800f>, line 1)